In [1]:
#Import part
import pandas as pd
import yfinance as yf
import numpy as np
import pandas_ta as ta
import time
import glob
from datetime import datetime, timedelta

In [7]:
"""
#In this section, we will try to find out correct TA numbers for strategy development.
# we can make a new row for each day and complete as a dataframe
# <<<================= Just update here ==========================>>>
bist1 = ["BIMAS.IS"]
endDate   = "2024-08-04"
startDate = "2024-05-01"
# <<<================= Just update the upper part ================>>> 

def date_based_test(startDate,endDate,bist1):
    stock_dict = {}
    stock_data = []
    stock_number = 0

    #Data has to download one time & we have to use it part by part
    for stocks in bist1:
        try:
            ticker = yf.Ticker(stocks)
            stock_info = ticker.info
            data = {key: stock_info.get(key, None) for key in ["symbol", "priceToBook", "currentPrice", "targetHighPrice", "targetLowPrice", "targetMeanPrice", "targetMedianPrice",
                                                                "bookValue", "open", "dayLow", "dayHigh", "recommendationKey", 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh']}
            #historical_data = ticker.history(period="1y")  # Get historical data for the stock
            #Date Test for spesific scores
            historical_data = ticker.history(start=startDate, end=endDate)  # Get historical data for the stock
            stock_dict[stocks] = historical_data
            if historical_data.empty:
                print(f"No historical data for {stocks}, skipping...")
                continue  # Skip this ticker if no historical data is found
            stock_number = 1 + stock_number
            #print(f"\r{stock_number}/{len(bist1)} Downloaded {stocks} data", end='', flush=True)

            historical_data['RSI'] = ta.rsi(historical_data['Close'], length=14)  # Calculate RSI (Relative Strength Index)
            adx_data = ta.adx(historical_data['High'], historical_data['Low'], historical_data['Close'], length=14)  # Calculate ADX (Average Directional Index)
            historical_data['ADX'] = adx_data['ADX_14']

            # Calculate MACD using EMA
            macd = ta.macd(historical_data['Close'], fast=12, slow=26, signal=9)
            historical_data = pd.concat([historical_data, macd], axis=1)

            # Calculate MACDAS (MACD Histogram)
            historical_data['MACDAS'] = historical_data['MACD_12_26_9'] - historical_data['MACDs_12_26_9']

            # Calculate MACDAS Signal Line (9-period EMA of MACDAS)
            historical_data['MACDAS_Signal'] = historical_data['MACDAS'].ewm(span=9, adjust=False).mean()

            historical_data['CCI'] = ta.cci(historical_data['High'], historical_data['Low'], historical_data['Close'], length=20)  # Calculate CCI (Commodity Channel Index)
            historical_data['ROC'] = ta.roc(historical_data['Close'], length=12)  # Calculate ROC (Rate of Change)
            historical_data['ATR'] = ta.atr(historical_data['High'], historical_data['Low'], historical_data['Close'], length=14)  # Calculate ATR (Average True Range)

            # Calculate Bollinger Bands
            bollinger = ta.bbands(historical_data['Close'], length=20, std=2)
            historical_data['BB_Middle'] = bollinger.iloc[:, 1]  # Middle Band
            historical_data['BB_Upper'] = bollinger.iloc[:, 2]  # Upper Band
            historical_data['BB_Lower'] = bollinger.iloc[:, 0]  # Lower Band
            historical_data['BB_BWidth'] = bollinger.iloc[:, 3]  # Bandwidth
            historical_data['BB_%B'] = bollinger.iloc[:, 4]  # %B

            # Calculate BB Signal
            recent_close = historical_data['Close'].iloc[-1]
            recent_bb_upper = historical_data['BB_Upper'].iloc[-1]
            recent_bb_lower = historical_data['BB_Lower'].iloc[-1]

            if recent_close > recent_bb_upper:
                bb_signal = "Sell"
            elif recent_close < recent_bb_lower:
                bb_signal = "Buy"
            else:
                bb_signal = "Neutral"

            data["BB_Signal"] = bb_signal

            historical_data['OBV'] = ta.obv(historical_data['Close'], historical_data['Volume'])  # Calculate OBV (On-Balance Volume)
            historical_data['CMF'] = ta.cmf(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'], length=20)  # Calculate CMF (Chaikin Money Flow)
            historical_data['AD'] = ta.ad(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'])  # Calculate A/D Line (Accumulation/Distribution Line)

            high_9 = historical_data['High'].rolling(window=9).max()  # Calculate Ichimoku Cloud components
            low_9 = historical_data['Low'].rolling(window=9).min()
            historical_data['Tenkan_sen'] = (high_9 + low_9) / 2
            high_26 = historical_data['High'].rolling(window=26).max()
            low_26 = historical_data['Low'].rolling(window=26).min()
            historical_data['Kijun_sen'] = (high_26 + low_26) / 2
            historical_data['Senkou_Span_A'] = ((historical_data['Tenkan_sen'] + historical_data['Kijun_sen']) / 2).shift(26)
            high_52 = historical_data['High'].rolling(window=52).max()
            low_52 = historical_data['Low'].rolling(window=52).min()
            historical_data['Senkou_Span_B'] = ((high_52 + low_52) / 2).shift(26)
            historical_data['Chikou_Span'] = historical_data['Close'].shift(-26)

            # Buy/Sell Signal Calculation for Ichimoku
            recent_close = historical_data['Close'].iloc[-1]
            recent_tenkan = historical_data['Tenkan_sen'].iloc[-1]
            recent_kijun = historical_data['Kijun_sen'].iloc[-1]
            recent_senkou_a = historical_data['Senkou_Span_A'].iloc[-1]
            recent_senkou_b = historical_data['Senkou_Span_B'].iloc[-1]

            
            if recent_close > max(recent_senkou_a, recent_senkou_b) and recent_tenkan > recent_kijun:
                data["IchiSignal"] = "Buy"
            elif recent_close < min(recent_senkou_a, recent_senkou_b) and recent_tenkan < recent_kijun:
                data["IchiSignal"] = "Sell"
            else:
                data["IchiSignal"] = "Neutral"

            data.update({"Tenkan_sen": recent_tenkan, "Kijun_sen": recent_kijun, "Senkou_Span_A": recent_senkou_a, "Senkou_Span_B": recent_senkou_b,
                         "Chikou_Span": historical_data['Chikou_Span'].iloc[-1], "RSI": historical_data['RSI'].iloc[-1], "ADX": historical_data['ADX'].iloc[-1],
                         "CCI": historical_data['CCI'].iloc[-1], "ROC": historical_data['ROC'].iloc[-1], "ATR": historical_data['ATR'].iloc[-1], "OBV": historical_data['OBV'].iloc[-1],
                         "CMF": historical_data['CMF'].iloc[-1], "AD": historical_data['AD'].iloc[-1],
                         "MACD": historical_data['MACD_12_26_9'].iloc[-1],  # MACD Line
                         "MACD_signal": historical_data['MACDs_12_26_9'].iloc[-1],  # MACD Signal Line
                         "MACD_Hist": historical_data['MACDh_12_26_9'].iloc[-1],  # MACD Histogram
                         "MACDAS": historical_data['MACDAS'].iloc[-1],  # MACDAS
                         "MACDAS_Signal": historical_data['MACDAS_Signal'].iloc[-1],  # MACDAS Signal Line
                         "BB_Middle": historical_data['BB_Middle'].iloc[-1],
                         "BB_Upper": historical_data['BB_Upper'].iloc[-1],
                         "BB_Lower": historical_data['BB_Lower'].iloc[-1],
                         'BB_%B': historical_data['BB_%B'].iloc[-1],
                         'BB_BWidth': historical_data['BB_BWidth'].iloc[-1],
                         "closingPrice": historical_data["Close"].iloc[-1]   
                         })
            stock_data.append(data)

        except Exception as e:
            print(f"Error fetching data for {stocks}: {e}")

    # Create a DataFrame from the stock data
    df = pd.DataFrame(stock_data)
    df["MACDAS-dif"] = df["MACDAS"] - df["MACDAS_Signal"] 
    df["change"] = ((df["currentPrice"] / df["open"]) - 1) * 100
    df["BB_Pot"] = ((df['BB_Upper'] / df["currentPrice"]) - 1) * 100
    df["BB_Opt"] = ((df['BB_Lower'] / df["currentPrice"]) - 1) * 100
    df["TrendWay"] = np.select([(df["ADX"] > 20) & (df["ROC"] > 0), (df["ADX"] > 20) & (df["ROC"] <= 0), (df["ADX"] <= 20)], ["upper", "lower", "no-trend"], default="unknown")

    # Removed Columns "BB_Signal",'BB_Middle','BB_Upper','BB_Lower', "Senkou_Span_A","IchiSignal", "Senkou_Span_B", "Tenkan_sen", "Kijun_sen","OBV", "CMF", "AD", "TrendWay","priceToBook","BB_Upper","BB_Middle", "BB_Lower" "BB_Pot","BB_Opt",
    df1 = df[["symbol","MACDAS-dif","MACDAS","MACDAS_Signal", 'BB_BWidth', 'BB_%B', 'fiftyTwoWeekLow', "closingPrice", 'fiftyTwoWeekHigh', "RSI", "ADX", "CCI", "ROC", "ATR"]]
    #removed for backtesting ["change",]
    df2 = df1.rename(columns={"fiftyTwoWeekHigh": "YHigh", "fiftyTwoWeekLow": "YLow"})
    df2["date"] = endDate
    pd.set_option('display.float_format', '{:.2f}'.format)
    #print("\nDownload process is done!")
    df2
    now = datetime.now()  # This part will copy for our sell lists. Especially further analysis.
    formatted_time = now.strftime("%m-%d_%H-%M-%S")
    df2.to_csv(f"CSVs/Backtesting/{stocks}_{endDate}.csv")
    df2
"""

'\n#In this section, we will try to find out correct TA numbers for strategy development.\n# we can make a new row for each day and complete as a dataframe\n# <<<================= Just update here ==========================>>>\nbist1 = ["BIMAS.IS"]\nendDate   = "2024-08-04"\nstartDate = "2024-05-01"\n# <<<================= Just update the upper part ================>>> \n\ndef date_based_test(startDate,endDate,bist1):\n    stock_dict = {}\n    stock_data = []\n    stock_number = 0\n\n    #Data has to download one time & we have to use it part by part\n    for stocks in bist1:\n        try:\n            ticker = yf.Ticker(stocks)\n            stock_info = ticker.info\n            data = {key: stock_info.get(key, None) for key in ["symbol", "priceToBook", "currentPrice", "targetHighPrice", "targetLowPrice", "targetMeanPrice", "targetMedianPrice",\n                                                                "bookValue", "open", "dayLow", "dayHigh", "recommendationKey", \'fiftyTwoWeekL

In [7]:
#// ... existing code ...
bist1 = ["BIMAS.IS"]
endDate   = "2024-08-04"
startDate = "2024-05-01"

# Data downloading and processing cell
def date_based_test(startDate,endDate,bist1):
    stock_dict = {}
    stock_data = []
    stock_number = 0
    for stocks in bist1:
        try:
            ticker = yf.Ticker(stocks)
            stock_info = ticker.info
            data = {key: stock_info.get(key, None) for key in ["symbol", "dayLow", "dayHigh", "recommendationKey", 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh']}
            #historical_data = ticker.history(period="1y") #"priceToBook", "currentPrice", "targetHighPrice", "targetLowPrice", "targetMeanPrice", "targetMedianPrice","bookValue", "open", 
            historical_data = ticker.history(start=startDate, end=endDate)  # Get historical data for the stock
            stock_dict[stocks] = historical_data
            if historical_data.empty:
                print(f"No historical data for {stocks}, skipping...")
                continue

            stock_number = 1 + stock_number
            #print(f"\r{stock_number}/{len(bist)} Downloaded {stocks} data", end='', flush=True)

            # Create a strategy with verified indicators
            MyStrategy = ta.Strategy(
                name="Core Indicators",
                description="Strategy using core technical indicators",
                ta=[
                    # Momentum Indicators
                    {"kind": "rsi"},
                    {"kind": "macd"},
                    {"kind": "cci"},
                    {"kind": "roc"},
                    {"kind": "stoch"},
                    {"kind": "willr"},
                    {"kind": "mom"},

                    # Trend Indicators
                    {"kind": "adx"},
                    {"kind": "ema", "length": 20},
                    {"kind": "sma", "length": 20},
                    {"kind": "tema"},

                    # Volatility Indicators
                    {"kind": "bbands"},
                    {"kind": "atr"},
                    {"kind": "natr"},
                    {"kind": "kc"},

                    # Volume Indicators
                    {"kind": "obv"},
                    {"kind": "cmf"},
                    {"kind": "mfi"},
                    {"kind": "vwap"},
                    {"kind": "ad"},

                    # Oscillator Indicators
                    {"kind": "ppo"},
                    {"kind": "stochrsi"}
                ]
            )

            # Calculate all indicators
            historical_data.ta.strategy(MyStrategy)

            # Get the most recent values for all indicators
            latest_data = historical_data.iloc[-1].to_dict()

            # Update data dictionary with all indicator values
            data.update({
                key: latest_data[key] 
                for key in latest_data.keys() 
                if key not in ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
            })

            stock_data.append(data)

        except Exception as e:
            print(f"\nError fetching data for {stocks}: {e}")

    # Create DataFrame and add derived calculations
    df = pd.DataFrame(stock_data)

    # Add custom calculations
    #df["change"] = ((df["currentPrice"] / df["open"]) - 1) * 100
    # Save results
    now = datetime.now()
    formatted_time = now.strftime("%m-%d_%H-%M-%S")
    #df.to_csv(f"CSVs/Backtesting/{bist1[0]} {formatted_time}.csv")
    with open(f'CSVs/Backtesting/Total.csv', 'a', newline='') as f:
        df.to_csv(f, header=False, index=False)

    # Display results
    pd.set_option('display.float_format', '{:.2f}'.format)
    #print("\nDownload process is done!")
    df

#// ... existing code ...

In [8]:
# we have to develop a dataframe for this

bist1 = ["TTRAK.IS"]
#endDate   = "2024-11-24"
endDate   = "2025-03-30"
spanDate  = "2025-02-01" 
startDate = "2024-09-01" #It must be 2 months before the span date

def generate_date_range(start_date: str, end_date: str) -> list:
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    date_list = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end - start).days + 1)]
    return date_list
endDateList = generate_date_range(spanDate, endDate)

for endDate in endDateList:
    time.sleep(0.3)
    date_based_test(startDate= startDate, endDate= endDate,bist1=bist1)
    print(f"{endDate} was downloaded.")
print("Process is done!")

2025-02-01 was downloaded.
2025-02-02 was downloaded.


KeyboardInterrupt: 

In [12]:
#Gather all CSV files in the current directory
csv_files = glob.glob("CSVs/Backtesting/*.csv")
now = datetime.now()
now = now.strftime("%m-%d_%H-%M-%S")
#Read and combine all files into a single DataFrame
df_combined = pd.concat(
(pd.read_csv(f) for f in csv_files),
ignore_index=True
)
#Export the combined data to a new CSV file
#df_combined.to_csv(f"CSVs/Combine/{now}combined.csv", index=False)
df_combined.to_csv(f"CSVs/Combine/{bist1[0]}_combined_{now}.csv", index=False)
print("Process is done!")


Process is done!
